In [ ]:
# Importing Googe Drive
from google.colab import drive
import sys, os
drive.mount('/content/drive/')

sys.path.append('/content/drive/Shareddrives/CFA project/utils')

In [ ]:
# Importing necessary packages and libraries
# Niu: added the first line to update openpyxl, which is necessary for pandas to read databases

!pip install openpyxl --upgrade

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
import statsmodels.api as sm
from functools import reduce

# Data Loading

#### Load quarterly data and select stocks that exist across all data

In [ ]:
"""
Jan
In this block of code I am importing all of the stock data from excel to pandas dataframes.
Each file is its own data frame and the naming goes as follows: "Q[quarter_number]Y[year]".
I excluded the last file so below I import the data from Q1Y2014 to Q4Y2020.
"""

quarter = 1
path_to_stock_data = "/content/drive/Shareddrives/CFA project/data/stocks"
for year in range(2014,2021):
  for i in range(4):
    if i+quarter == 4:
      globals()['Q%sY%s' % (i+1,year)] = pd.read_excel(path_to_stock_data+'/'+str(year+((i+quarter-1)//4))+'Q'+str(i+quarter)+'.xlsx')
    else:
      globals()['Q%sY%s' % (i+1,year)] = pd.read_excel(path_to_stock_data+'/'+str(year+((i+quarter-1)//4))+'Q'+str((i+quarter)%4)+'.xlsx')



In [ ]:
"""
  select stocks that exist in all dataset
"""

quarters = []
# NOTE: The Q1Y2014 dataswet includes a ton of null values in the interest expense column. We might want to drop it.
for year in range(2014,2021):
  for quarter in range(1,5):
    df = "Q%sY%s" % (quarter,year)
    globals()[df]['date'] = str(year) + 'Q' + str(quarter)
    quarters.append(df)

d1 = eval(quarters[0])
intersection_set = set(d1['Stock'])

for quarter in quarters:
    df = eval(quarter) # this line allows me to access the dataset using its name, which is in a string format.
    df_stocks = set(df['Stock'])
    intersection_set = intersection_set.intersection(df_stocks)

print('Number of stocks: ', len(intersection_set))

Number of stocks:  715


#### Read macro data and industrial categories

In [ ]:
"""

  Read macro data

"""

macros_features = ['CPIAUCSL', 'PAYEMS', 'PPIACO', 'T10YIE', 'T5YIE','UNRATE']
macros_list = []
for feature in macros_features:
  path_to_file = '/content/drive/Shareddrives/CFA project/data/macros/' + feature + '.csv'
  feature_df = pd.read_csv(path_to_file)
  feature_df[feature] = pd.to_numeric(feature_df[feature], errors='coerce') 
  feature_df = feature_df.dropna()
  feature_df['DATE'] = pd.PeriodIndex(feature_df['DATE'], freq='Q')
  feature_df = feature_df.sort_values('DATE').groupby('DATE').first()
  macros_list.append(feature_df)

macro_df = reduce(lambda x, y: pd.merge(x, y, on = 'DATE', how='inner'), macros_list)
macro_df = macro_df.reset_index()
macro_df['DATE'] = macro_df['DATE'].astype('str')


In [ ]:
#@title Retrieve industry information
""" 
Jonathan, Jan

Using package "yfinance" to retrieve the industry information from each stock info
The code only serves for the purpose of display, as it takes 10s to retrieve each stock's info. (7000s in total)

Way to load the data is given. 

Please not run these lines, but the next code box with only two lines!!!




industry_dict = {} # pairs of {stock name: industry} will be stored to a dictionary

for i in range(len(intersection_set)): 
    name_str = list(intersection_set)[i]
    name_str = name_str[:name_str.find('.')]
    temp_stock = yf.Ticker(name_str) 

    if("industry" in temp_stock.info.keys()): 
        industry_dict[list(intersection_set)[i]] = temp_stock.info['industry']
    else: # cases when there is no industry info of some stocks
        industry_dict[list(intersection_set)[i]] = None

### To store retrieved information into a file 
np.save('industry.npy', industry_dict) 


"""

' \nJonathan, Jan\n\nUsing package "yfinance" to retrieve the industry information from each stock info\nThe code only serves for the purpose of display, as it takes 10s to retrieve each stock\'s info. (7000s in total)\n\nWay to load the data is given. \n\nPlease not run these lines, but the next code box with only two lines!!!\n\n\n\n\nindustry_dict = {} # pairs of {stock name: industry} will be stored to a dictionary\n\nfor i in range(len(intersection_set)): \n    name_str = list(intersection_set)[i]\n    name_str = name_str[:name_str.find(\'.\')]\n    temp_stock = yf.Ticker(name_str) \n\n    if("industry" in temp_stock.info.keys()): \n        industry_dict[list(intersection_set)[i]] = temp_stock.info[\'industry\']\n    else: # cases when there is no industry info of some stocks\n        industry_dict[list(intersection_set)[i]] = None\n\n### To store retrieved information into a file \nnp.save(\'industry.npy\', industry_dict) \n\n\n'

In [ ]:
#@title Stores sub-industry names (Just run it!)

CommunicationServices = ["Advertising Agencies", "Broadcasting", "Electronic Gaming & Multimedia", "Entertainment", "Farm & Heavy Construction Machinery", "Publishing", "Telecom Services"]

Industrials = ["Aerospace & Defense", "Airlines", "Airports & Air Services", "Apparel Manufacturing", "Business Equipment & Supplies", "Conglomerates", "Consulting Services", "Electrical Equipment & Parts", "Engineering & Construction", "Integrated Freight & Logistics", "Marine Shipping", "Metal Fabrication","Pollution & Treatment Controls", "Railroads", "Security & Protection Services", "Specialty Business Services", "Specialty Industrial Machinery", "Staffing & Employment Services", "Trucking", "Waste Management"]

ConsumerDiscretionary = ["Apparel Retail", "Auto Manufacturers", "Auto Parts", "Auto & Truck Dealerships","Computer Hardware",
"Consumer Electronics", "Department Stores", "Discount Stores", "Education & Training Services", "Footwear & Accessories", "Furnishings, Fixtures & Appliances", "Grocery Stores", "Home Improvement Retail", "Internet Retail", "Leisure", "Lodging", "Luxury Goods","Personal Services", "Recreational Vehicles", "Resorts & Casinos", "Restaurants", "Specialty Retail", "Tobacco", "Tools & Accessories", "Travel Services"]

Materials = ["Aluminum", "Agricultural Inputs", "Building Materials", "Building Products & Equipment", "Chemicals", "Copper", "Gold", "Other Industrial Metals & Mining", "Packaging & Containers", "Paper & Paper Products", "Specialty Chemicals", "Steel"]

Financials = ["Asset Management", "Capital Markets", "Credit Services", "Financial Data & Stock Exchanges", "Insurance Brokers"]

ConsumerStaples = ["Beverages—Brewers", "Beverages—Non-Alcoholic", "Confectioners", "Farm Products", "Food Distribution", "Household & Personal Products", "Packaged Foods", "Pharmaceutical Retailers"]

HealthCare = ["Biotechnology", "Diagnostics & Research", "Drug Manufacturers—General", "Drug Manufacturers—Specialty & Generic","Health Information Services","Healthcare Plans","Medical Care Facilities", "Medical Devices", "Medical Instruments & Supplies", "Medical Distribution"]

InformationTechnology = ["Communication Equipment", "Electronic Components", "Electronics & Computer Distribution", "Information Technology Services", "Internet Content & Information","Scientific & Technical Instruments", "Semiconductor Equipment & Materials", "Semiconductors", "Software—Application", "Software—Infrastructure"]

RealEstate = ["Industrial Distribution", "REIT—Healthcare Facilities", "REIT—Hotel & Motel", "REIT—Specialty", "Real Estate Services", "Rental & Leasing Services"]

Energy = ["Oil & Gas Drilling", "Oil & Gas E&P","Oil & Gas Equipment & Services", "Oil & Gas Integrated","Oil & Gas Midstream", "Oil & Gas Refining & Marketing"]

Utilities = ["Solar", "Utilities—Diversified", "Utilities—Independent Power Producers", "Utilities—Regulated Electric", "Utilities—Regulated Gas", "Utilities—Regulated Water", "Utilities—Renewable"]

industry_names = ["CommunicationServices","Industrials","ConsumerDiscretionary","Materials","Financials","ConsumerStaples","HealthCare","InformationTechnology","RealEstate","Energy","Utilities"]

In [ ]:
""" 

read industry associated with each stock 

"""

industry_dict = np.load('/content/drive/Shareddrives/CFA project/data/industrial/industry.npy',allow_pickle='TRUE').item()

In [3]:
''' 

Jonathan

Classified all the companies into 12 categories (including 1 "Not Applicable" class)

'''

## To load the industry information
industry_dict = np.load('/content/drive/Shareddrives/CFA project/data/industrial/industry.npy',allow_pickle='TRUE').item()

## To determine how many different sectors are there

industry_list = [industry_dict[xi] for xi in industry_dict]
for i in range(len(industry_list)):
  if(industry_list[i]==None):
    industry_list[i] = "Not Applicable"
  for j in range(len(industry_names)):
    if(industry_list[i] in eval(industry_names[j])):
      industry_list[i] = industry_names[j]
      break 
                               
# Currently thinking about how to classify them into fewer categories
# Probably classifying them based on Global Industry Classification Standard (GICS)
# https://www.msci.com/our-solutions/indexes/gics (⌘+Click)

unique, counts = np.unique(np.array(industry_list), return_counts=True)
dict(zip(unique, counts)) # 117 industry classes are there in total


NameError: ignored

In [2]:
len(industry_list)

NameError: ignored

In [ ]:
#@title Creating a dictionary called avg_sharpe_ratio where for each stock I will include its average sharpe ratio over the 6 year time period
"""
Jan

Creating a dictionary called avg_sharpe_ratio where for each stock I will include its average sharpe ratio over the 6 year time period
"""

import matplotlib.pyplot as plt
import numpy as np

avg_sharpe_ratio = {}
for quarter in stocks:
  data = eval(quarter)

  #data.set_index("Stock", inplace=True)

  for c in intersection_set:
    row = data.loc[(data['Stock'] == c)]
    val = row['3M.Sharpe.Ratio'].tolist()
    if c not in avg_sharpe_ratio:
      avg_sharpe_ratio[c] = val[0]/28
    else:
      avg_sharpe_ratio[c] += val[0]/28


x = []
y = []
id = 1
for key in avg_sharpe_ratio:
  x.append(id)
  y.append(avg_sharpe_ratio[key])
  id += 1

fig, ax = plt.subplots(figsize=(24,8))

ax.scatter(x,y)
plt.title("Stock's average Sharpe ratio over the 6 year period")
ax.set(ylim=(-0.2, 1.5), yticks=np.arange(-0.2, 1.5,0.1))
plt.show()



#print(avg_sharpe_ratio) # printing the entire dictionary to see the results


NameError: ignored

#### get stock years since ipo, untill the corresponding year on each row

In [ ]:
!pip install yahoo_fin
from yahoo_fin.stock_info import *

In [ ]:
tickers = [x.split(".")[0] for x in list(df['Stock'].unique())] 

tickers_ipo = {}
for ticker, orig_ticker in zip(tickers,list(df['Stock'].unique())):
  try:
    ipo_date = get_data(ticker, interval = '1mo').sort_index().iloc[0].name.year
    tickers_ipo[orig_ticker] = ipo_date
  except:
    tickers_ipo[orig_ticker] = None
    
tickers_ipo

#### Combine quarterly data into one dataframe

In [ ]:
"""
Sammy

  Combine quarterly data into one dataframe

"""

df = pd.DataFrame()
for quarter in quarters:
  quarter_data = globals()[quarter]
  df = df.append(globals()[quarter])
df = df[df['Stock'].isin(intersection_set)]

df = df.sort_values(['date'])
df.reset_index(inplace=True)

# merge with macro data
df = df.merge(macro_df, left_on='date', right_on='DATE', how='left')
df = df.drop(columns=['date', 'index'])

# industry category
df['industry'] = df['Stock'].map(industry_dict)

# years since ipo
df['age'] = df['DATE'].map(lambda x: x.split('Q')[0]).astype('int') - df['Stock'].map(tickers_ipo)
df.info()

In [ ]:
df['Month'] = df['DATE'].astype(str).str[-1].astype(int)

In [ ]:
from helpers import encode_cyclic
df = encode_cyclic(df, 'Month', 4, inplace = False)

In [ ]:
ax = df.plot.scatter('Month_sin', 'Month_cos').set_aspect('equal')

In [ ]:
df.to_pickle("/content/drive/Shareddrives/CFA project/data/loaded_df.pkl")

# Data pre-processing

In [ ]:
!pip install category_encoders

from category_encoders.target_encoder import TargetEncoder
from category_encoders.sum_coding import SumEncoder
from category_encoders.m_estimate import MEstimateEncoder
from category_encoders.leave_one_out import LeaveOneOutEncoder
from category_encoders.helmert import HelmertEncoder
from category_encoders.cat_boost import CatBoostEncoder
from category_encoders.james_stein import JamesSteinEncoder
from category_encoders.one_hot import OneHotEncoder

from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_pickle('/content/drive/Shareddrives/CFA project/data/loaded_df.pkl')
df.info()

#### calculate corresponding y label


*   X: Values -> y: Stock price
*   X: %change -> y: Stock price diff (next quarter - current quarter)



In [ ]:
fundamentals = ['Free.Operating.Cash.Flow', 'Total.Debt.Outstanding',
       'Total.Common.Shares.Outstanding',
       'Cash.and.Equivalents', 'Net.Income', 'Interest.Expense',
       'Short.Interest', 'Avg.Daily.Volume.30D', 'Current.Assets',
       'Non.Current.Assets', 'Current.Liabilities']

technicals = ['RSI.Exponential','MACD.Line1', 'Bollinger.Band,Width', 'Price.Mo.Volatility.T12M']

macros_features = ['CPIAUCSL', 'PAYEMS', 'PPIACO', 'T10YIE', 'T5YIE','UNRATE']

In [ ]:
# keep two types of data: X: [attributes] -> y: close price
#                         X: [pct_change_attributes] -> y: pct_change in close price
#TODO: determine y: average? EWA? One single close may not show the whole level of increase/decrease
df = df.sort_values(by=['DATE'])
df_val = df.copy()
df_val[['next_close', 'next_sharpe']] = df_val.groupby(['Stock'])['Price.Close', '3M.Sharpe.Ratio'].transform(lambda x: x.shift(-1))[['Price.Close', '3M.Sharpe.Ratio']]
df_val = df_val.dropna()
df_diff = df.copy()

# calculate pct_diff as y
df_diff['pct_change_close'] = df_diff.groupby(['Stock'])['Price.Close'].transform(lambda x: x.pct_change().shift(-1))

# fundamental, close price, and macro shall be saved as pct_change to reflect the pct_change in y; 
#TODO: shall technical indicators be counted as pct_change as well or remain as is?
features_as_pct = fundamentals + ['Price.Close'] + macros_features
df_diff[features_as_pct] = df_diff.groupby(['Stock'])[features_as_pct].transform(lambda x: x.pct_change())[features_as_pct]
df_diff = df_diff.dropna()

In [ ]:
df_val = df_val.dropna()
df_diff = df_diff.dropna()

In [ ]:
df_diff.info()

In [ ]:
df_diff = df_diff.sample(frac=1)
X = df_diff.drop(columns=['Stock', 'DATE', 'pct_change_close'])
y = df_diff['pct_change_close']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 40)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
CBE_encoder = CatBoostEncoder()

In [ ]:
X_train['industry'] = CBE_encoder.fit_transform(X_train['industry'], y_train)

In [ ]:
X_test['industry'] = CBE_encoder.transform(X_test['industry'])

In [ ]:
df_diff = df_diff.drop(columns=['Stock'])
df_val = df_val.drop(columns=['Stock'])
df = df.drop()

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize': (11,8)})
sns.heatmap(X_train.corr())

In [ ]:
col_2_drop = ['T5YIE', 'Price.Close', 'PAYEMS']
X_train = X_train.drop(columns=col_2_drop)